<a href="https://colab.research.google.com/github/EOREUA/EOREUA/blob/main/023_converting_PDFs_to_excel_for_accounting_SW_ver_corrected.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%shell
sudo apt-get -y remove ocrmypdf
sudo apt-get -y update
sudo apt-get -y install \
    ghostscript \
    icc-profiles-free \
    liblept5 \
    libxml2 \
    pngquant \
    python3-cffi \
    python3-distutils \
    python3-pkg-resources \
    python3-reportlab \
    qpdf \
    tesseract-ocr \
    zlib1g \
    unpaper

Reading package lists... Done
Building dependency tree       
Reading state information... Done
Package 'ocrmypdf' is not installed, so not removed
The following packages were automatically installed and are no longer required:
  cuda-command-line-tools-10-0 cuda-command-line-tools-10-1
  cuda-command-line-tools-11-0 cuda-compiler-10-0 cuda-compiler-10-1
  cuda-compiler-11-0 cuda-cuobjdump-10-0 cuda-cuobjdump-10-1
  cuda-cuobjdump-11-0 cuda-cupti-10-0 cuda-cupti-10-1 cuda-cupti-11-0
  cuda-cupti-dev-11-0 cuda-documentation-10-0 cuda-documentation-10-1
  cuda-documentation-11-0 cuda-documentation-11-1 cuda-gdb-10-0 cuda-gdb-10-1
  cuda-gdb-11-0 cuda-gpu-library-advisor-10-0 cuda-gpu-library-advisor-10-1
  cuda-libraries-10-0 cuda-libraries-10-1 cuda-libraries-11-0
  cuda-memcheck-10-0 cuda-memcheck-10-1 cuda-memcheck-11-0 cuda-nsight-10-0
  cuda-nsight-10-1 cuda-nsight-11-0 cuda-nsight-11-1 cuda-nsight-compute-10-0
  cuda-nsight-compute-10-1 cuda-nsight-compute-11-0 cuda-nsight-compute-

In [1]:
!pip3 install ocrmypdf

In [2]:
!pip install pdfplumber -q

     |████████████████████████████████| 46 kB 2.5 MB/s 
     |████████████████████████████████| 139 kB 12.9 MB/s 


In [3]:
import os

import requests
import pdfplumber

In [4]:
def download_file(url):
    local_filename = url.split('/')[-1]
    
    with requests.get(url) as r:
        assert r.status_code == 200, f'error, status code is {r.status_code}'
        with open(local_filename, 'wb') as f:
            f.write(r.content)
        
    return local_filename

In [5]:
invoice = 'https://bit.ly/2UJgUpO'
invoice_pdf = download_file(invoice)

In [6]:
invoice_pdf

'2UJgUpO'

In [7]:
with pdfplumber.open(invoice_pdf) as pdf:
    page = pdf.pages[0]
    text = page.extract_text()
    print(text)

In [8]:
os.system(f'ocrmypdf {invoice_pdf} output.pdf')

0

In [9]:
with pdfplumber.open('output.pdf') as pdf:
    page = pdf.pages[0]
    text = page.extract_text(x_tolerance=2)
    print(text)

Stanford Plumbing & Heating INVOICE
123 Madison drive, Seattle, WA, 7829Q
plumbingstanford.com
990-120-4560 ?
BILLTO Invoice No: *PINVO2081
Allen Smith Invoice Date: nawe
87 Private st, Seattle, WA Due Date: 12/01/18
allen@gmail.com
990-302-1898
DESCRIPTION QTY/ HR UNIT PRICE TOTAL
Installed new kitchen sink (hours) 3 $0.00 150.00
Toto sink 1 $00.00 00.00
Worcester greenstar magnetic system fier 1 190.00 190.00
Nest smart thermostat 1 250.00 250.00
Worcester Greenstar 301 1 1800.00 1500.00
‘SUBTOTAL 2590.00
OFSCOUNT ‘$0.00
SUBTOTAL LESS DISCOUNT 2540.00
‘Tank you for your business!
‘TAX RATE 12.00%
TOTAL TAX 304.80
Balance Due $2,844.80
Terms& instructions
Please pay within 20 days by PayPal (bob@stanfordplumbingcom)
Installed products have S year warranty


In [10]:
lines = text.split('\n')

In [ ]:
lines

['Stanford Plumbing & Heating INVOICE',
 '123 Madison drive, Seattle, WA, 7829Q',
 'plumbingstanford.com',
 '990-120-4560 ?',
 'BILLTO Invoice No: *PINVO2081',
 'Allen Smith Invoice Date: nawe',
 '87 Private st, Seattle, WA Due Date: 12/01/18',
 'allen@gmail.com',
 '990-302-1898',
 'DESCRIPTION QTY/ HR UNIT PRICE TOTAL',
 'Installed new kitchen sink (hours) 3 $0.00 150.00',
 'Toto sink 1 $00.00 00.00',
 'Worcester greenstar magnetic system fier 1 190.00 190.00',
 'Nest smart thermostat 1 250.00 250.00',
 'Worcester Greenstar 301 1 1800.00 1500.00',
 '‘SUBTOTAL 2590.00',
 'OFSCOUNT ‘$0.00',
 'SUBTOTAL LESS DISCOUNT 2540.00',
 '‘Tank you for your business!',
 '‘TAX RATE 12.00%',
 'TOTAL TAX 304.80',
 'Balance Due $2,844.80',
 'Terms& instructions',
 'Please pay within 20 days by PayPal (bob@stanfordplumbingcom)',
 'Installed products have S year warranty']

In [ ]:
import re

In [ ]:
amt_re = re.compile(r'\.\d\d$')

In [ ]:
subt = 0

for line in lines:
    if 'SUBTOTAL' in line:
        break
    if amt_re.search(line):
        subt += float(line.split()[-1].replace(',', '').replace('$', ''))

In [ ]:
subt

2090.0

In [ ]:
lines

['Stanford Plumbing & Heating INVOICE',
 '123 Madison drive, Seattle, WA, 7829Q',
 'plumbingstanford.com',
 '990-120-4560 ?',
 'BILLTO Invoice No: *PINVO2081',
 'Allen Smith Invoice Date: nawe',
 '87 Private st, Seattle, WA Due Date: 12/01/18',
 'allen@gmail.com',
 '990-302-1898',
 'DESCRIPTION QTY/ HR UNIT PRICE TOTAL',
 'Installed new kitchen sink (hours) 3 $0.00 150.00',
 'Toto sink 1 $00.00 00.00',
 'Worcester greenstar magnetic system fier 1 190.00 190.00',
 'Nest smart thermostat 1 250.00 250.00',
 'Worcester Greenstar 301 1 1800.00 1500.00',
 '‘SUBTOTAL 2590.00',
 'OFSCOUNT ‘$0.00',
 'SUBTOTAL LESS DISCOUNT 2540.00',
 '‘Tank you for your business!',
 '‘TAX RATE 12.00%',
 'TOTAL TAX 304.80',
 'Balance Due $2,844.80',
 'Terms& instructions',
 'Please pay within 20 days by PayPal (bob@stanfordplumbingcom)',
 'Installed products have S year warranty']